# 2.1: Finite Elements on a Pac-Man Mesh
### Sam Reynolds, 2023

This example demonstrates how to set up and solve a finite element problem on a
punctured mesh. 
The model problem under consideration is a simple diffusion-reaction problem
\begin{align*}
	-\nabla\cdot(a \, \nabla u) + c \, u &= f \quad \text{in } \Omega, \\
	u &= 0 \quad \text{on } \partial\Omega,
\end{align*}
where $a, c$ are constant scalars and $f$ is a polynomial.
The associated weak form is
\begin{align*}
	\int_\Omega a \, \nabla u \cdot \nabla v \, dx
	+ \int_\Omega c \, u \, v \, dx
	&= \int_\Omega f \, v \, dx
	\quad \forall v \in H^1_0(\Omega).
\end{align*}
In previous examples, we saw that we can evalate these integrals on each cell
$K$ in a mesh $\mathcal{T}$ of the domain $\Omega$, provided that $u$ and $v$
are elements of a *local Poisson space* $V_p(K)$.
We define the *global Poisson space* $V_p(\mathcal{T})$ as the space of
continuous functions in $H^1_0(\Omega)$ whose restriction to each cell $K$ is
an element of $V_p(K)$.
By constructing a basis $\{\phi_1, \dots, \phi_N\}$ of $V_p(\mathcal{T})$ by 
continuously "stitching" the local basis functions together,
we seek a finite element solution $\tilde{u} \in V_p(\mathcal{T})$ such that
\begin{align*}
	&\tilde{u} = \sum_{i=1}^N u_i \, \phi_i,
	\\
	&\int_\Omega a \, u_i \nabla \phi_i \cdot \nabla \phi_j \, dx
	+ \int_\Omega c \, u_i \, \phi_i \, \phi_j \, dx
	= \int_\Omega f \, \phi_j \, dx
\end{align*}
Let's set a few parameters before we go any further. 
`deg` is the polynomial degree of global Poisson space,
`n` is edge sampling parameter (as used in previous examples).

**(!) WARNING:** 
Higher order spaces (`deg > 3`) are still under development.

In [ ]:
deg = 1
n = 64

## Mesh construction
The mesh we will use for this example was constructed in 
[Example 0.1](ex0.1-mesh-building.ipynb).
For convenience, the same mesh can be constructed by calling the `pacman_mesh`
function in the `mesh.meshlib` module.

In [ ]:
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np

import puncturedfem as pf

T = pf.meshlib.pacman_subdiv()
# T = pf.meshlib.pacman()

Let's take a look at the mesh by using the `MeshPlot` class.

In [ ]:
pf.plot.MeshPlot(T.edges, n).draw(show_axis=False, pad=0.0)

## Build global function space 
The global function space $V_p(\mathcal{T})\subset H^1(\Omega)$ 
is the space of continuous functions such that each function belongs to 
$V_p(K)$ when restricted to any cell $K\in\mathcal{T}$.
(Note that we use `deg` to denote the polynomial degree $p$.)

To proceed with the computation, we define the quadrature scheme(s) used to 
parameterize the edges of the mesh.

In [ ]:
quad_dict = pf.get_quad_dict(n)

The global function space `V` is built from the mesh `T`, along with the `deg`
parameter and the information necessary to parameterize the edges of the mesh.

In [ ]:
V = pf.GlobalFunctionSpace(T, deg, quad_dict)

## Examine basis functions on a cell

In [ ]:
# cell_idx = 8
# vpk = V.build_local_function_space(cell_idx, verbose=True, compute_interior_values=True)

In [ ]:
# print(vpk.num_funs)

In [ ]:
# for j, v in enumerate(vpk.get_basis()):
#     pf.plot.LocalFunctionPlot(v).draw(show_colorbar=False, show_axis=False, fill=True, filename=f"out/ghost-basis-p{deg}-{v.key.fun_type}{v.key.glob_idx}.pdf")

## Define a bilinear form
The bilinear form 
\begin{align*}
	B(u,v) = 
	\int_\Omega a \, \nabla u \cdot \nabla v ~dx
	+ \int_\Omega c \, u \, v ~dx
\end{align*}
and the right-hand side linear functional
\begin{align*}
	F(v) = \int_\Omega f \, v ~dx
\end{align*}
are declared as follows,
with `diffusion_coefficient` $a = 1$, 
`reaction_coefficient` $c = 1$,
and `rhs_poly` $f(x) = 1 \cdot x^{(0, 0)}$.

In [ ]:
a = 1.0
c = 1.0
f = pf.Polynomial([(1.0, 0, 0)])

B = pf.BilinearForm(
    diffusion_constant=a,
    reaction_constant=c,
    rhs_poly=f,
)

print(B)

## Set up the finite element solver
A finite element solver needs two things: the global function space and the bilinear form. 

In [ ]:
solver = pf.Solver(V, B, compute_interior_values=False)

## Examine the finite element matrix

In [ ]:
A = solver.glob_mat.todense()
cond_num = np.linalg.cond(A)
print(f"dof = {V.num_funs}, cond_num = {cond_num:.4e}")

| p | dof | condition number | Jacobi |
|---|-----|------------------|--------|
| 1 | 38  | 6.7901e+02 | 1.2801e+02 |
| 2 | 84  | 3.1345e+06 | 3.2949e+02 |
| 3 | 142 | 6.8565e+10 | 7.5502e+05 |
| 4 | 219 | 2.4033e+17 | 2.3654e+27 |

In [ ]:
plt.figure()
plt.imshow(A)
plt.colorbar()

In [ ]:
plt.figure()
log10_abs_A = np.log10(np.abs(A))
plt.imshow(log10_abs_A)
plt.colorbar()

In [ ]:
plt.figure()
eigenvalues = np.linalg.eigvalsh(A)
plt.plot(eigenvalues, "o-")
plt.grid(True)

In [ ]:
print(eigenvalues[abs(eigenvalues) < 0.1])

## Reduced system (zero Dirichlet BCs)

In [ ]:
non_boudary_indices = set()
for abs_cell_idx in range(V.mesh.num_cells):
    for key in V.cell_dofs[abs_cell_idx]:
        if not key.is_on_boundary:
            non_boudary_indices.add(key.glob_idx)
non_boudary_indices = sorted(non_boudary_indices)

A_reduced = A[non_boudary_indices, :][:, non_boudary_indices]
cond_num_reduced = np.linalg.cond(A_reduced)
print(f"dof = {A_reduced.shape[0]}, cond_num = {cond_num_reduced:.4e}")

In [ ]:
plt.figure()
plt.imshow(A_reduced)
plt.colorbar()

plt.figure()
log10_abs_A_reduced = np.log10(np.abs(A_reduced))
plt.imshow(log10_abs_A_reduced)
plt.colorbar()

In [ ]:
eigenvalues_reduced = np.linalg.eigvalsh(A_reduced)
plt.figure()
plt.plot(eigenvalues_reduced, "o-")
plt.grid(True)

print(eigenvalues_reduced[abs(eigenvalues_reduced) < 0.1])

## Jacobi preconditioner

In [ ]:
# Jacobi preconditioner
D = np.diag(np.diag(A))
M = np.linalg.inv(D)
A_precond = M @ A
cond_num_precond = np.linalg.cond(A_precond)
print(f"dof = {V.num_funs}, cond_num = {cond_num_precond:.4e}")

plt.figure()
plt.imshow(A_precond)
plt.colorbar()

plt.figure()
log10_abs_A_precond = np.log10(np.abs(A_precond))
plt.imshow(log10_abs_A_precond)
plt.colorbar()

eigenvalues_precond = np.linalg.eigvalsh(A_precond)
plt.figure()
plt.plot(eigenvalues_precond, "o-")
plt.grid(True)

print(eigenvalues_precond[abs(eigenvalues_precond) < 0.1])

In [ ]:
# preconditioned reduced system
D = np.diag(np.diag(A_reduced))
M = np.linalg.inv(D)
A_reduced_precond = M @ A_reduced
cond_num_reduced_precond = np.linalg.cond(A_reduced_precond)
print(f"dof = {A_reduced.shape[0]}, cond_num = {cond_num_reduced_precond:.4e}")

plt.figure()
plt.imshow(A_reduced_precond)
plt.colorbar()

plt.figure()
log10_abs_A_reduced_precond = np.log10(np.abs(A_reduced_precond))
plt.imshow(log10_abs_A_reduced_precond)
plt.colorbar()

eigenvalues_reduced_precond = np.linalg.eigvalsh(A_reduced_precond)
plt.figure()
plt.plot(eigenvalues_reduced_precond, "o-")
plt.grid(True)

print(eigenvalues_reduced_precond[abs(eigenvalues_reduced_precond) < 0.1])

## Solve the finite element system
To solve the system we worked hard to set up, we can call the `solve()` method
on the `Solver` object.

In [ ]:
solver.solve()

## Compute the $H^1$ error
The $H^1$ error is computed with 
\begin{align*}
    \|u - \tilde{u}\|_{H^1(\Omega)}^2
    = \|u\|_{H^1(\Omega)}^2 - \|\tilde{u}\|_{H^1(\Omega)}^2
\end{align*}
where $\tilde{u}$ is the finite element solution and $u$ is the exact solution.
Computations with *Mathematica* give
\begin{align*}
    \|u\|_{H^1(\Omega)}^2
    \approx 0.257592478386321945
\end{align*}
correct to all digits shown.

In [ ]:
u_h1_sq = 0.257592478386321945

In [ ]:
S = solver.stiff_mat
M = solver.mass_mat
alpha = solver.soln
u_tilde_h1_sq = alpha @ S @ alpha + alpha @ M @ alpha
h1_error_sq = u_h1_sq - u_tilde_h1_sq
h1_error = np.sqrt(np.abs(h1_error_sq))
print(deg, h1_error)

| p | $\|u - \tilde{u}\|_{H^1(\Omega)}$ | ratio |
|---|-----------------------------------|-------|
| 1 | 0.22515505298089233 | n/a |
| 2 | 0.035927410348521906 | 6.266943561941293 |
| 3 | 0.01110922569722671 | 3.2340157025966954 |

In [ ]:
errors = [
    0.22515505298089233,
    0.035927410348521906,
    0.01110922569722671,
    0.0006352311617389737,
    0.004537523786757339,
]
num_errors = len(errors)
for p in range(1, num_errors):
    print(p + 1, errors[p - 1] / errors[p])

## Plot the solution
We can visualize the solution by 
creating an instance of the `GlobalFunctionPlot` class.
There are two types of plots available: 
a conventional contour plot (`fill=False`)
or a heat map (`fill=True`).
To view the figure in this notebook, set `show_fig = True`.
To save it to a file, set the `filename` keyword argument in the 
`draw()` method.

In [ ]:
# global_plot = pf.plot.GlobalFunctionPlot(solver)

In [ ]:
# global_plot.draw(filename=f"../doc/logo/pacman.svg")
# print(global_plot.global_range)

In [ ]:
# global_plot.draw(
#     plot_type="grad_x1",
#     use_interp=False,
#     filename=f"out/pac-man-p{deg}-grad-x1.pdf",
# )
# print(global_plot.global_grad1_range)

## Plot the solution on a cell

In [ ]:
# cell_idx = 8
# v = global_plot.global_function[cell_idx]
# local_plot = pf.plot.LocalFunctionPlot(v)
# local_plot.draw()

In [ ]:
# local_plot.draw(show_triangulation=True)

## Plot a global basis function
Let's take a look at one of the global basis functions.

In [ ]:
# import numpy as np

# idx = 5
# coef = np.zeros(V.num_funs)
# coef[idx] = 1.0
# pf.plot.GlobalFunctionPlot(solver, coef).draw()